In [1]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install nltk

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
# the imports used in A2
import re
# import json
from glob import glob
import os
from io import StringIO
from itertools import groupby
import pickle

import numpy as np
# import bs4
# %matplotlib inline
# import matplotlib.pyplot as plt
# Imports that might help with various functionality
import functools
import operator

# Additional imports from A3
from __future__ import print_function
import math
from collections import defaultdict
# from nltk.tokenize import TreebankWordTokenizer
# import Levenshtein  # package python-Levenshtein

# Additional imports from A5
import nltk
# nltk.download()
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import cPickle

[nltk_data] Downloading package stopwords to /Users/Pegah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Do not run these 2 blocks -- Pickling

In [2]:
# SPACY package
import spacy
# nlp = spacy.load('en')
nlp = spacy.load('en_core_web_sm')

In [4]:
import pickle
with open("nlp.pickle", "wb+") as f:
    pickle.dump(nlp, f)

### Run this instead

In [2]:
pkl_nlp = open('nlp.pickle', 'rb')
nlp = pickle.load(pkl_nlp)

IOError: [Errno 2] No such file or directory: 'nlp.pickle'

# Import Dataset

In [3]:
import json
with open("data.json", "r") as f:
    women_summaries = json.load(f)

print("Each woman's entry is a dictionary with the following keys...")
print(women_summaries[0].keys())

Each woman's entry is a dictionary with the following keys...
[u'url', u'views', u'name', u'summary']


In [7]:
# REMOVED FROM SPACY VERSION
# creates a list of women to keep as a global variable
women_names = list()
for i in range(len(women_summaries)):
    name = women_summaries[i]['name']
    end_index = name.find('(')
    if end_index != -1 and name[: (end_index-1)] not in women_names :
        women_names.append(name[: (end_index-1)])
    elif end_index == -1 and name not in women_names :
        women_names.append(name)
        
# print(women_names)
# women_names is a list of the names of women in order of the JSON file

# Deduplicate

In [19]:
check_names = []
deduped_women = list(women_summaries)
for woman in deduped_women:
    if woman['name'] not in check_names:
        check_names.append(woman['name'])
    else:
        deduped_women.remove(woman)
import pickle
pickle.dump(deduped_women, open( "deduped_women.pickle", "wb" ), protocol=2)
pkl_file_wmn = open('deduped_women-search.pickle', 'rb')
deduped_women = pickle.load(pkl_file_wmn)

In [36]:
pkl_file_wmn = open('deduped_women-search.pickle', 'rb')
deduped_women = pickle.load(pkl_file_wmn)

# Spacy Dictionary
For building token dictionary for W2W matrix

In [7]:
from __future__ import unicode_literals

In [11]:
# Builds a dictionary that maps woman to the list of tokens associated with her
# {'woman1' : ['token1', 'token2', ... 'tokenn']}
def build_spacy_token_dictionary(input_summaries, input_women_names):
    # Builds a dictionary that maps each woman to their tokenized words (yes we've already done this but we want the spacy version)
    women_token_dictionary = dict()
    
    for i in range(len(input_women_names)):
        name = input_women_names[i]
        summary = input_summaries[i]['summary']
    
        index_was = summary.find('was ')
        index_is = summary.find('is ')
        index_currently = summary.find('currently ')
        if (index_was == -1 and index_is == -1):
            summary2 = summary[index_currently:]
        elif (index_was == -1 and index_currently == -1):
            summary2 = summary[index_is:]
        elif (index_is == -1 and index_currently == -1):
            summary2 = summary[index_was:]
        else :
            index_min = min(index_is, index_was)
            summary2 = summary[index_min:]
            
        # lowercase long string of summary
        summary_lower = summary2.lower()
        # doc object type
        summary_nlp = nlp(summary_lower)
        # type list
        summary_list = [token.text for token in summary_nlp]
        # type list, but without stop words
        summary_token_filtered = [w for w in summary_list if w not in stopwords.words('english')]
        
        # convert to a string separated by spaces
        summary_str = " ".join(summary_token_filtered) 
        summary_nlp = nlp(summary_str)
        women_token_dictionary[name] = summary_nlp
    
#     print(women_token_dictionary)
    return women_token_dictionary

In [12]:
# COMMENTED OUT BECAUSE TOKENIZER DOES THIS
women_token_dictionary = build_spacy_token_dictionary(women_summaries, women_names)
# # print(women_token_dictionary)

# TF-IDF Vectorizer
#### Includes pickling the vectorizer and the TF-IDF matrix

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words="english", max_df = 0.95)
matx = vectorizer.fit_transform(map(lambda x: x["summary"], deduped_women))

import pickle
with open("myvectorizer.pickle", "wb+") as f:
    pickle.dump(vectorizer, f)
    pickle.dump(matx, f)

# Spacy Similarity Matrix

In [38]:
def build_sim_matrix(input_token_dictionary, input_len_names, input_women_names):
    # Builds matrix using spacy that stores each woman's cosine similarity to each other
    
    sim_matrix = np.zeros(shape = (input_len_names, input_len_names))
    
    for woman1 in input_token_dictionary:
        for woman2 in input_token_dictionary:
            woman1_index = input_women_names.index(woman1)
            woman2_index = input_women_names.index(woman2)
#             print(input_token_dictionary[woman1])
#             print(input_token_dictionary[woman2])
            sim_matrix[woman1_index][woman2_index] = input_token_dictionary[woman1].similarity(input_token_dictionary[woman2])
    np.fill_diagonal(sim_matrix, 0)
    
    return sim_matrix
        

In [15]:
women2women_cosine_sim_matrix = build_sim_matrix(women_token_dictionary, len(women_names), women_names)


In [16]:
import operator

# Code taken from A5
def get_ranked_women(input_woman, input_sim_matrix, input_women_names) :
#     print(input_sim_matrix)
    '''Return sorted rankings (most to least similar) of women as 
        a list of two-element tuples, where the first element is the 
        woman's name and the second element is the similarity score
    '''
    
    # Get index from woman's name
    idx = input_women_names.index(input_woman)
    
    # Get list of similarity scores for woman
    score_lst = input_sim_matrix[idx]
    women_score_lst = [(input_women_names[index], score) for index, score in enumerate(score_lst)]
    
    # Do not account for woman herself in ranking
    women_score_lst = women_score_lst[:idx] + women_score_lst[idx+1:]
    
    # Sort rankings by score (most similar to least similar)
    women_score_lst = sorted(women_score_lst, key=lambda x: -x[1])
    
    # Only returning top 5!
    return women_score_lst[:5]

In [17]:
# Builds a 'top 5 most similar' list of woman for every woman
def create_top_5_dict_women(input_sim_mat, num_women, input_women_names):
    women_top5 = {}
    
    # Loop through each woman
    for woman in range(num_women):
        similarity_list = get_ranked_women(input_women_names[woman], input_sim_mat, input_women_names)
        similarity_list = [x[0] for x in similarity_list]
        women_top5[input_women_names[woman]] = similarity_list
        
    return women_top5
            

In [18]:
top5_dict_women2women = create_top_5_dict_women(women2women_cosine_sim_matrix, len(women_names), women_names)

In [19]:
top5_dict_women2women['Rachel Kuske']

[u'Olabisi Ugbebor',
 u'Anne Greenbaum',
 u'H\xe9l\xe8ne Esnault',
 u'Anu\u0161ka Ferligoj',
 u'Marina Ratner']

In [20]:
pickle.dump(top5_dict_women2women, open( "top5_dict_women2women.pickle", "wb" ), protocol=2)

# Query output for Cossim

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle

# Load from the pickle
pkl_file = open('myvectorizer.pickle', 'rb')
vectorizer = pickle.load(pkl_file)
matx = pickle.load(pkl_file)

def cossim_scores(vectorizer, matx, query, data_dict):
    q_vec = vectorizer.transform([query])
    sim_doc_scores = cosine_similarity(q_vec, matx)
    
    
    '''
    This next block is commented out because we are changing this function to return cosine sim scores
    '''
#     sim_docs = np.argsort(sim_doc_scores.flatten())[::-1]
#     return_docs = []
#     for hit in sim_docs:
#         if sim_doc_scores[0][hit] > 0:
# #             print(sim_doc_scores[0][hit])
#             return_docs.append(data_dict[hit])
#     if len(return_docs)>30:
#         return_docs = return_docs[:30]

    '''
    Alright! back to business
    '''
    return sim_doc_scores[0]

In [40]:
sum(cossim_scores(vectorizer, matx, u"lalala", deduped_women))

0.0

# Making our Spacy Matrix

## <span style="color:red">Important!</span>
The next block takes a while to run, but it's been pickled! Don't run the next 2 code blocks. Instead, just compile spacy_sim and run the block after it.

In [41]:
# DO NOT RUN!!
spacy_array = np.zeros((len(deduped_women),384))
for index, woman in enumerate(deduped_women):
#     print(index)
    wom_vec = nlp(woman['summary']).vector
    if len(wom_vec) != 384:
        wom_vec = np.zeros(384)
    spacy_array[index] = wom_vec

In [42]:
# pickle here
# DON'T RUN THIS EITHER!
with open("spacyarray.pickle", "wb+") as f:
    pickle.dump(spacy_array, f)

In [43]:
def spacysim_scores(spacy_mat, query, data_dict):
    '''
    spacy_mat: numpy array of documents by spacy vectors
    
    query: query string
    
    RETURNS
    
    '''
    q = nlp(query)
    sim_scores = np.dot(spacy_mat, q.vector)/((np.linalg.norm(spacy_mat)*np.linalg.norm(q.vector))+1)
    
    '''
    commenting out this next part because we are changing this to return sim scores
    '''
#     sim_docs = np.argsort(sim_scores)[::-1]
#     return_docs = []
#     for hit in sim_docs[0:30]:
#         return_docs.append(data_dict[hit])

    return sim_scores

In [44]:
pkl_file_spacy = open('spacyarray.pickle', 'rb')
spacy_array = pickle.load(pkl_file_spacy)

spacysim_scores(spacy_array, u"lalala", deduped_women)

array([0.00543635, 0.00578734, 0.00486329, ..., 0.00678422, 0.00482919,
       0.00523706])

# Combining our sim measures
Spacy similarity scores + cosine similarity scores

In [45]:
def return_query(cossim_arr, spacysim_arr, data_dict):
    '''
    cossim_arr : 1D array of cosine similarity scores
    spacysim_arr: 1D array of spacy similarity scores
    data_dict: list of dicts (women)
    
    returns: TUPLE!
    
    return_docs : list of dict with top 30 women (keys are name, summary, views)
    
    '''
    cosine_used = True
    spacy_used = True
    if max(cossim_arr) == 0:
        cosine_used = False
    if max(spacysim_arr) == 0:
        spacy_used = False
    weighted_scores = cossim_arr*0.7 + spacysim_arr*0.3
    print(sum(weighted_scores))
    sim_docs = np.argsort(weighted_scores)[::-1]
    return_docs = []
    for hit in sim_docs[0:30]:
        print(hit)
        print(data_dict[hit])
        return_docs.append(data_dict[hit])
    return return_docs, cosine_used, spacy_used

In [46]:
query = u"researched cancer"
s = spacysim_scores(spacy_array, query, deduped_women)
print(sum(s))
c = cossim_scores(vectorizer, matx, query, deduped_women)
print(sum(c))

20.392535167727555
5.788407403991879


In [47]:
# return_query(c, s, deduped_women_wack)
print(deduped_women_wack[100])
print(deduped_women[44])

{u'name': u'Lai-Sang Young', u'summary': u'Lai-Sang Lily Young (Chinese: \u694a\u9e97\u7b19, born 1952) is a mathematician from Hong Kong, who holds the Henry & Lucy Moses Professorship of Science and is a professor of mathematics and neural science at the Courant Institute of Mathematical Sciences of New York University. Her research interests include dynamical systems, ergodic theory, chaos theory, probability theory, statistical mechanics, and neuroscience. She is particularly known for introducing the method of Markov returns in 1998, which she used to prove exponential correlation delay in Sinai billiards and other hyperbolic dynamical systems.'}
{u'name': u'Laura Person', u'summary': u'Laura J. Person is an American mathematician specializing in low-dimensional topology. She is a Distinguished Teaching Professor of Mathematics at the State University of New York at Potsdam.'}
